# Подготовка данных для обучения и тестирования моделей

- импорт

In [1]:
import pandas
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import pickle
import numpy as np

In [2]:
df = pandas.read_csv("train_games_DS.csv")
df.head()

,game_number,crash,total_bank
S,t,o,p
700001,1.03,235.25,NaN
700002,1.61,5.33,NaN
700003,7.32,-68.03,NaN
700004,2.19,-50.07,NaN


- обработка данных

In [3]:
df = df[df.total_bank!='p'] # убираем S T O P

In [4]:
#формирование обработанных DF

gn = list(df.game_number.index)
cr = list(map(float,list(df.game_number)))
tb = list(map(float,list(df.crash)))
dic = {'game_number': gn, 'crash': cr, 'total_bank': tb} 
dfr = pandas.DataFrame(dic)

- подготовка df, на которых модель будет обучаться и тестироваться

In [5]:
def riffle(lst1, lst2):
    if not lst1:
        return lst2
    elif not lst2:
        return lst1
    return lst1[0:1] + riffle(lst2, lst1[1:])

In [6]:
# обучение будет происходить на основе предыдущих 49 игр
past = 49
future = 1
start = past
end = len(dfr) - future

In [7]:
dfr

,game_number,crash,total_bank
0,700001,1.03,235.25
1,700002,1.61,5.33
2,700003,7.32,-68.03
3,700004,2.19,-50.07
4,700005,1.03,218.15
...,...,...,...
42430,742461,3.14,-46.87
42431,742462,7.42,-22.28
42432,742463,6.33,-11.92
42433,742464,1.00,80.74


In [8]:
past_list = []
future_list = []

for i in range(start, end):
    past_data_cr = cr[(i - past):i] # Данные за последние 49 игр по crash
    future_data_cr = cr[i:(i + future)] # Данные за следующую игру по crash
    past_data_tb = tb[(i - past):i] # Данные за последние 49 игр по total_bank 

    forappend = riffle(list( map( float, past_data_cr)), list( map( float, past_data_tb)))
    past_list.append(forappend)
    future_list.append(list( map( float, future_data_cr)))

    # списки заголовков
past_columns = [] 
for i in range(2*past):
    past_columns.append(f"past_{i}")

future_columns = []
for i in range(future):
    future_columns.append(f"future_{i}")
    
    # формирование DF
X = pandas.DataFrame(past_list, columns=past_columns) # Входные данные (49 игр в прошлом)
Y = pandas.DataFrame(future_list, columns=future_columns)# Выходные данные (игра в будущем)

In [9]:
X.head()

,past_0,past_1,past_2,past_3,past_4,past_5,past_6,past_7,past_8,past_9,...,past_88,past_89,past_90,past_91,past_92,past_93,past_94,past_95,past_96,past_97
0,1.03,235.25,1.61,5.33,7.32,-68.03,2.19,-50.07,1.03,218.15,...,1.22,63.21,1.27,136.82,2.16,-87.96,1.01,184.50,1.97,-40.65
1,1.61,5.33,7.32,-68.03,2.19,-50.07,1.03,218.15,23.20,-57.83,...,1.27,136.82,2.16,-87.96,1.01,184.50,1.97,-40.65,2.44,-104.81
2,7.32,-68.03,2.19,-50.07,1.03,218.15,23.20,-57.83,1.50,-13.54,...,2.16,-87.96,1.01,184.50,1.97,-40.65,2.44,-104.81,1.04,382.35
3,2.19,-50.07,1.03,218.15,23.20,-57.83,1.50,-13.54,1.12,138.61,...,1.01,184.50,1.97,-40.65,2.44,-104.81,1.04,382.35,1.39,83.81
4,1.03,218.15,23.20,-57.83,1.50,-13.54,1.12,138.61,2.89,-69.76,...,1.97,-40.65,2.44,-104.81,1.04,382.35,1.39,83.81,2.51,-123.07


In [10]:
Y.head()

,future_0
0,2.44
1,1.04
2,1.39
3,2.51
4,1.37


In [ ]:
#записываем в отдельные csv файлы
X.to_csv('trainDS_X.csv') 
Y.to_csv('trainDS_Y.csv')